In [1]:
# 필요한 패키지 import
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import pandas as pd
import lxml.html
from selenium.webdriver import Chrome, Firefox
import time
from selenium import webdriver
import os

In [2]:
# 엑셀 저장 함수 정의
def save_excel(df):
    excel = pd.ExcelWriter('../data/BlueHouse.xlsx')
    df.to_excel(excel, '.', index=False)
    excel.save()

In [4]:
# 전역 변수 선언 (전체 레코드가 병합될 데이터프레임, 전체 레코드가 저장될 리스트, 각 레코드를 저장할 임시 리스트)
df = pd.DataFrame({'번호':[], '분류':[], '제목':[], '만료일':[], '참여인원':[]})
my_list = []
temp_str_list = ['' for x in range(5)]

In [5]:
# 크롬드라이버 권한 설정 변경 (크롬드라이버는 구글에서 다운로드 가능)
os.chmod('../chromedriver', 755)

In [6]:
# 셀레니움 웹드라이버 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized");

In [7]:
# 웹드라이버 브라우저 열기
browser = webdriver.Chrome('/Users/jaehyeob/venv_02/CrawlingScraping/chromedriver', options=options)

In [10]:
# 1 ~ 20 페이지 크롤링 (하나의 세션에서 5분 내에 25페이지를 초과해서 접근할 경우 세션 에러 발생)
# 따라서, 5분 간격으로 1~20, 21~40, 41~60, 61~80, 81~100 페이지 따로 크롤링 진행하였음
page_num = 1

while page_num <= 20:
    url = 'https://www1.president.go.kr/petitions/?c=0&only=1&page={}&order=1'.format(page_num)

    # 해당 url 열기
    browser.get(url)
    # 로딩 시간 고려해서 대기 시간 설정
    browser.implicitly_wait(5)

    # xpath 사용해서 tag 검색으로 elements 불러오기
    lis = browser.find_elements_by_xpath('//*[@id="cont_view"]/div[2]/div/div/div[2]/div[2]/div[4]/div/div[2]/div[2]/ul/li')
    
    # 모든 element에 대하여
    for li in lis:
        # tag 사이의 text를 개행 기준으로 나눠서 임시 리스트에 저장
        temp_str_list = li.text.split('\n')
        # 임시 리스트를 전체 리스트에 추가
        my_list.append(pd.DataFrame({'번호':[temp_str_list[0]], '분류':[temp_str_list[1]], '제목':[temp_str_list[2]], 
                                     '만료일':[temp_str_list[3]], '참여인원':[temp_str_list[4]]}))
            
    page_num = page_num + 1

In [11]:
# 전체 데이터프레임으로 병합
df = pd.concat(my_list)
# 인덱스 재정렬
df.reset_index(drop=True, inplace=True)

In [16]:
# 전체 데이터프레임 엑셀로 저장
save_excel(df)